In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox

pd.options.display.max_rows = 100

In [2]:
def lower_case_column_names(x):
    x.columns=[i.lower() for i in x.columns]
    return x

In [3]:
file1 = pd.read_csv("Data/file1.csv")
file1 = lower_case_column_names(file1)
file1.rename(columns={"st":"state", "customer lifetime value":"customer_ltv", "monthly premium auto":"premium", "number of open complaints":"open_complaints", "policy type":"policy", "vehicle class":"vehicle_class", "total claim amount":"total_claim"}, inplace = True)
# file1.head()

In [4]:
file2 = pd.read_csv("Data/file2.csv")
file2 = lower_case_column_names(file2)
file2.rename(columns={"st":"state", "customer lifetime value":"customer_ltv", "monthly premium auto":"premium", "number of open complaints":"open_complaints", "policy type":"policy", "vehicle class":"vehicle_class", "total claim amount":"total_claim"}, inplace=True)
file2 = file2[["customer", "state", "gender", "education", "customer_ltv", "income", "premium", "open_complaints", "policy", "vehicle_class", "total_claim"]]
# file2.head()

In [5]:
file3 = pd.read_csv("Data/file3.csv")
file3 = lower_case_column_names(file3)
file3.rename(columns={"customer lifetime value":"customer_ltv", "monthly premium auto":"premium", "number of open complaints":"open_complaints", "policy type":"policy", "vehicle class":"vehicle_class", "total claim amount":"total_claim"}, inplace=True)
file3 = file3[["customer", "state", "gender", "education", "customer_ltv", "income", "premium", "open_complaints", "policy", "vehicle_class", "total_claim"]]
# file3.head()

In [6]:
def combining_files():
    return pd.concat([file1,file2,file3], axis=0)

In [7]:
df = combining_files()
df.drop(["customer"], axis=1, inplace=True)
# df

In [8]:
# print("state -> ", df["state"].unique())
# print("gender -> ", df["gender"].unique())
# print("education -> ", df["education"].unique())
# print("policy type -> ", df["policy"].unique())
# print("vehicle class -> ", df["vehicle_class"].unique())

In [9]:
df['state'] = df['state'].replace(['AZ', 'WA', 'Cali'],['Arizona', 'Washington', 'California'])
# df["state"].value_counts()

In [10]:
df['gender'] = df['gender'].replace(['Femal', 'female'], 'F')
df['gender'] = df['gender'].replace(['Male'], 'M')
# df['gender'].value_counts()

In [11]:
df['education'] = df['education'].replace(['Bachelors'], 'Bachelor')
# df['education'].unique()
# df['education'].value_counts()

In [12]:
df['customer_ltv'] = df['customer_ltv'].apply(lambda x: float(x.strip("%"))/100 if isinstance(x,str) else x)
df['customer_ltv'] = pd.to_numeric(df['customer_ltv'], errors='coerce')
df['customer_ltv'] = df['customer_ltv'].apply(lambda x: float(round(x, 0)) if isinstance(x, (int, float)) else x)

In [13]:
df['open_complaints'] = df['open_complaints'].apply(lambda x: float(x.split('/')[1]) if isinstance(x, str) else x)
# df['open_complaints'].unique()

In [14]:
# original_df = df
df = df.drop_duplicates()

In [15]:
df = df.fillna(0)
# df

In [16]:
df['income'] = df['income'].replace([0], df.income.mean())
df['premium'] = df['premium'].replace([0], df.premium.mean())
df['customer_ltv'] = df['customer_ltv'].replace([0], df.customer_ltv.mean())
df['state'] = df['state'].replace([0], 'U')
df['gender'] = df['gender'].replace([0], "U")
# df

,state,gender,education,customer_ltv,income,premium,open_complaints,policy,vehicle_class,total_claim
0,Washington,U,Master,7980.553578,37707.283909,1000.0,0.0,Personal Auto,Four-Door Car,2.704934
1,Arizona,F,Bachelor,6980.000000,37707.283909,94.0,0.0,Personal Auto,Four-Door Car,1131.464935
2,Nevada,F,Bachelor,12887.000000,48767.000000,108.0,0.0,Personal Auto,Two-Door Car,566.472247
3,California,M,Bachelor,7646.000000,37707.283909,106.0,0.0,Corporate Auto,SUV,529.881344
4,Washington,M,High School or Below,5363.000000,36357.000000,68.0,0.0,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...
7064,California,F,College,4100.000000,47761.000000,104.0,0.0,Personal Auto,Four-Door Car,541.282007
7065,California,M,Bachelor,23406.000000,71941.000000,73.0,0.0,Personal Auto,Four-Door Car,198.234764
7067,California,M,Bachelor,8164.000000,37707.283909,85.0,3.0,Corporate Auto,Four-Door Car,790.784983
7068,California,M,College,7524.000000,21941.000000,96.0,0.0,Personal Auto,Four-Door Car,691.200000


In [17]:
# print("state -> ", df["state"].unique())
# print("gender -> ", df["gender"].unique())
# print("education -> ", df["education"].unique())
# print("customer_ltv -> ", df["customer_ltv"].unique())
# print("income -> ", df["income"].unique())
# print("premium -> ", df["premium"].unique())
# print("open_complaints -> ", df["open_complaints"].unique())
# print("policy -> ", df["policy"].unique())
# print("vehicle_class -> ", df["vehicle_class"].unique())
# print("total_claim -> ", df["total_claim"].unique())

In [18]:
df['state'] = df['state'].replace(['California', 'Oregon', 'Washington'],['West Region', 'North West', 'East'])
df['state'] = df['state'].replace(['Arizona', 'Nevada'], 'Central')
print("state ->\n", df["state"].value_counts())

state ->
 West Region    2996
Central        2506
North West     2480
East            724
U                 1
Name: state, dtype: int64


In [19]:
df = df.rename(columns={"state": "zones"})

In [20]:
df = df.applymap(lambda x:x.lower() if isinstance(x, str) else x)
# df = df.applymap(lambda x:x.lower() if type(x) == str else x)

In [21]:
df['customer_ltv'] = df['customer_ltv'].round(decimals=2)
df['income'] = df['income'].round(decimals=2)
df['premium'] = df['premium'].round(decimals=2)
df['total_claim'] = df['total_claim'].round(decimals=2)

In [22]:
df

,zones,gender,education,customer_ltv,income,premium,open_complaints,policy,vehicle_class,total_claim
0,east,u,master,7980.55,37707.28,1000.0,0.0,personal auto,four-door car,2.70
1,central,f,bachelor,6980.00,37707.28,94.0,0.0,personal auto,four-door car,1131.46
2,central,f,bachelor,12887.00,48767.00,108.0,0.0,personal auto,two-door car,566.47
3,west region,m,bachelor,7646.00,37707.28,106.0,0.0,corporate auto,suv,529.88
4,east,m,high school or below,5363.00,36357.00,68.0,0.0,personal auto,four-door car,17.27
...,...,...,...,...,...,...,...,...,...,...
7064,west region,f,college,4100.00,47761.00,104.0,0.0,personal auto,four-door car,541.28
7065,west region,m,bachelor,23406.00,71941.00,73.0,0.0,personal auto,four-door car,198.23
7067,west region,m,bachelor,8164.00,37707.28,85.0,3.0,corporate auto,four-door car,790.78
7068,west region,m,college,7524.00,21941.00,96.0,0.0,personal auto,four-door car,691.20
